In [2]:
from bs4 import BeautifulSoup
import csv

In [3]:
html_path = './FrontendSensai.html'

In [4]:
with open(html_path, 'r', encoding='utf-8') as file:
        soup = BeautifulSoup(file, 'html.parser')
        

In [5]:
content = []
    
# Assuming headers (like "Basic Structure of an HTML Document") are in <h2>, objectives in <p>, and steps in <ul>/<li>
sections = soup.find_all('h1')  # Change the tag based on your HTML structure
subsections = soup.find_all('h2')  # Change the tag based on your HTML structure

In [9]:
sections

[<h1 class="c12" id="h.prhdjdxy2tsw"><span class="c17">HTML</span></h1>,
 <h1 class="c12" id="h.122bvaevtc6v"><span class="c17">CSS</span></h1>,
 <h1 class="c12" id="h.wshwtvdoubxw"><span class="c17">JavaScript</span></h1>,
 <h1 class="c12" id="h.32ix1422dybs"><span class="c17">React</span></h1>]

In [7]:
sections[0].get_text()

'HTML'

In [9]:
section = sections[0]
section.find_next('h2')

<h2 class="c14" id="h.l93c8n62hx2m"><span class="c16 c7">HTML
            Introduction</span></h2>

In [10]:
subsection = section.find_next('h2')


In [13]:
task = subsection.find_next('h3')

In [14]:
task

<h3 id="h.mda3aq7rj990" style="display:inline"><span class="c5">Basic Structure of an HTML Document</span></h3>

In [20]:
p = task.find_next('p')

In [37]:
' '.join(p.get_text().split())

'Objective: Understand the basic structure of an HTML document.'

In [31]:
p.find_next().find_next()

<span class="c0"> Understand the basic structure of an HTML
            document.</span>

In [40]:
def norm(name):
    return ' '.join(name.split())

In [91]:
tasks = []


for section in sections:

    section_name = norm(section.get_text().strip())
    # print(f"Section: {section_name}")
    
    # Find all subsections (h2 tags) within this section
    subsections = section.find_all_next('h2')
    
    # We need to stop when we reach the next main section (h1 tag)
    next_section = section.find_next('h1')

    # if section.get_text() != 'React':
    #     continue

    if not subsections:
        subsections = [section]
    
    for subsection in subsections:
        # Break the loop if we've reached the next main section
        if next_section and subsection.find_previous('h1') != section:
            break

        subsection_name = norm(subsection.get_text())
        # print(f"  Subsection: {subsection_name}")

        # Find all h3 tags within this subsection
        h3_tags = []
        next_element = subsection.find_next()

        while next_element and next_element.name != 'h2' and next_element.name != 'h1':
            if next_element.name == 'h3':
                h3_tags.append(next_element)

            next_element = next_element.find_next()
        
        # Iterate through all p tags
        for h3 in h3_tags:
            task_name = norm(h3.get_text())
            # print(f"    Name: {task_name}")

            task = {
                'Name': task_name,
                'Description': '',
                'Tags': list(set([section_name, subsection_name])),
            }

            # if task_name != 'Book Review Display':
            #     continue

            # Find all p tags within this h3 tag
            p_tags = []
            next_element = h3.find_next()
            while next_element and next_element.name != 'h3' and next_element.name != 'h2' and next_element.name != 'h1':
                if next_element.name == 'p':
                    p_tags.append(next_element)
                next_element = next_element.find_next()
            
            # Iterate through all p tags
            has_text = False
            for p in p_tags:
                p_text = norm(p.get_text())
                if not p_text:
                    continue

                has_text = True

                task['Description'] = p_text
                # print(f"      {p_text}")

            # print('---------------------')

            if p_tags and has_text:
                # Find all ordered lists after this p tag before the next p tag or h3
                list_items = []
                next_element = p.find_next()
                while next_element and next_element.name != 'p' and next_element.name != 'h3' and next_element.name != 'h2' and next_element.name != 'h1':
                    if next_element.name in ['ol', 'ul']:
                        list_items.append((next_element.name, next_element))
                    next_element = next_element.find_next()

            else:
                list_items = []
                next_element = h3.find_next()
                while next_element and next_element.name != 'h3' and next_element.name != 'h2' and next_element.name != 'h1':
                    if next_element.name in ['ol', 'ul']:
                        list_items.append((next_element.name, next_element))
                    next_element = next_element.find_next()
                            
            
            # Iterate through all list items in the order they appear
            for list_type, list_element in list_items:
                if list_type == 'ol':
                    # print(f"      Ordered List:")
                    spacing = "\n\t"
                else:
                    # print(f"      Unordered List:")
                    spacing = "\n\t\t"

                # Check if the list contains an h3 tag
                if list_element.find('h3'):
                    # If h3 is found, not a part of this list
                    continue
                
                for li in list_element.find_all('li'):
                    li_text = norm(li.get_text())
                    task['Description'] += f"{spacing}- {li_text}"

            tasks.append(task)

    # print()  # Add a blank line for readability between sections

In [92]:
import pandas as pd

In [93]:
df = pd.DataFrame(tasks)

In [95]:
df.to_csv('FrontendSensai.csv')

In [29]:
next_element = p.find_next('ol')
ol_elements = []

while next_element and next_element.name != 'h2':
    if next_element.name == 'ol':
        ol_elements.append(next_element)
    next_element = next_element.find_next()

ol_elements

<ol class="c6 lst-kix_cy7qa9umb8yt-1" start="2">
<li class="c4 li-bullet-0"><span class="c0">Adding Metadata</span></li>
</ol>

In [1]:
for section in sections:
   name = section.get_text()
   description_tag = section.find_next('p')
   description = description_tag.get_text() if description_tag else ""
   
   # Find all the list items (assuming they contain the details)
   list_items = section.find_next('ul').find_all('li')
   details = " ".join([item.get_text() for item in list_items])

NameError: name 'sections' is not defined